In [2]:
!pip install wfdb
import wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### pan tompkins

In [7]:
import numpy as np
from scipy.signal import butter, filtfilt

def bandpass_filter(signal, fs, lowcut=5, highcut=15):
    nyq = 0.5 * fs
    b, a = butter(1, [lowcut/nyq, highcut/nyq], btype='band')
    return filtfilt(b, a, signal)

def pan_tompkins(ecg, fs):
    ecg = bandpass_filter(ecg, fs)
    derivative = np.diff(ecg)
    squared = derivative ** 2
    window_size = int(0.15 * fs)
    mwi = np.convolve(squared, np.ones(window_size)/window_size, mode='same')
    threshold = np.mean(mwi)
    r_peaks = np.where(mwi > threshold)[0]
    return r_peaks, mwi


In [21]:
import pandas as pd
import os

files_data_sheet = pd.read_csv('/content/drive/MyDrive/ECG PCG DATASET/labels.csv')
files_data_sheet

,Record Name,Label
0,ECGPCG0001,Exercise: pedaling a stationary bicycle
1,ECGPCG0013,Rest: laying on bed
2,ECGPCG0019_1,Slow walk
3,ECGPCG0019_2,Fast walk
4,ECGPCG0019_3,Sit down and stand up
...,...,...
68,ECGPCG0067,Exercise: bicycle stress test
69,ECGPCG0066,Exercise: bicycle stress test
70,ECGPCG0065,Exercise: bicycle stress test
71,ECGPCG0069,Exercise: bicycle stress test


In [38]:
output_dir = "/content/drive/MyDrive/ECG PCG DATASET/pan_tompkins_processed_records"
os.makedirs(output_dir, exist_ok=True)
files_names = files_data_sheet['Record Name'].values
for file_name in files_names:
  record = wfdb.rdrecord(f"/content/drive/MyDrive/ECG PCG DATASET/{file_name}")
  fs = record.fs
  ecg_signal = record.p_signal[:,0]
  pan_tompkins_processed_signal = pan_tompkins(ecg_signal, fs)[1]
  output_path = os.path.join(output_dir, f"{file_name}_processed.csv")
  np.savetxt(output_path, pan_tompkins_processed_signal, delimiter=",")